# Training and Making predicting

## Description

**8 Models considered:**

For Penelized linear models: Lasso, Ridge, Elastic Net regression 

For Linear dimension reduction tech: PCR, partial least squares regressions 

For Nonlinear models: RF, gradient boosted tree regression (GBR), gradient boosted tree regression with dropout (Dart), deep feed=forward neural networks (FFN)

**Assessing predictive power:**

standard out-of-sample $R^2$ statistic

$$
R^2_{OS} = 1 - \frac{\sum_{(i,t)\in T_3} (r_{i, s, t+1} 
            - \hat{r}_{i,s,t+1})^2} {\sum_{(i,t)\in T_3 r^2_{i, s,t+1}}}
$$

**Four Periods:**

$T_1$ (1996/01 -- 2000/12) -> training

$T_2$ (2001 -- 2002) -> *hyperparameter* optimization **validating**

$T_3$ (2003) -> **testing**

> We repeat this procedure for each year in the testing sample from 2003 through 2020, increasing the number of training years by one at each iteration.

That is: 
* First iteration: 1996 ~ 2000 as train, 2001 ~ 2002 as validate, 2003 as test 

* Second iteration: 1996 ~ 2001 as train, 2002 ~ 2003 as validate, 2004 as test 

... 

* i-th iteration: 1996 ~ 2000 + i-1 as train, 2000 + i ~ 2000 + i+1 as validation, 2000 + i+2 as test

**Cross sectional asset pricing test**

> In cross-sectional asset pricing tests, our main objective is not to forecast time-series variation in future returns, but rather cross-sectional return spreads in the testing sample.

Han et al (2021) proposes a cross-sectional out-of-sample $R^2$:

$$
R^2_{OS; XS} = 1 - \frac{\sum_{(i,t)\in T_3} [(r_{i,s,t+1} - \bar{r}_{i,s,t+1}) - (\hat{r}_{i,s,t+1} - \bar{\hat{r}}_{i,s,t+1})]^2} 
{\sum_{(i,t)\in T_3} (r_{i,s,t+1} - \bar{r}_{i,s,t+1})^2}
$$

> $R^2_{OS; XS}$ focuses on relative expected returns across options, for which accurate predictions result in profitable long-short trading strategies.

**Statistical significance of models forecasts**

**1. CW test (for individual model)**

$$
CW^{(j)} = \frac{\bar{c}^{(j)}}  {\hat{\sigma}_c^{(j)}}
$$

where $\bar{c}^{(j)}$ is the time series average, and $\hat{\sigma}_c^{(j)}$ is the Newey and West 
standard error ***of*** the mean difference squared forecast errors: 

$$
c_{t+1}^{(j)} = \frac{1}{n_{T_3}} \sum_{(i,t)\in T_3} [r_{i,s,t+1}^2 - (\hat{e}_{i,t+1}^{(j)})^2]
$$

where $n_{T_3}$ is the number of observations in the testing sample and $\hat{e}_{i,t+1}$ is the 
forecast error on option i at time t+1 for method j. 

**2. DM test (for comparison)**

Diebold and Mariano (1995) (DM) test

The DM test-statisics for a comparison between methods 1 and 2 is defined as: 

$$
DM^{(1,2)} = \frac{\bar{d}^{(1,2)}} {\hat{\sigma}_d^{(1,2)}}
$$

where $\bar{d}^{(1,2)}$ is the time series average, and $\hat{\sigma}_d^{(1,2)}$ is the Newey and West 
standard error ***of*** the mean difference squared forecast errors: 

$$
d^{(1,2)}_{t+1} = \frac{1}{n_{T_3}} \sum_{(i,t)\in T_3} [(\hat{e}^{(1)}_{i, t+1})^2 - (\hat{e}^{(2)}_{i, t+1})^2]
$$


In [138]:
def CW_test(true_pred_return):
    '''
    true_pred_return should be a data frame
    with columns = ['optionid', 'time', 'true_return', 'pred_return']
    '''
    c = 1 / true_pred_return.shape[0] * (
        true_pred_return.groupby(["optionid", "time"]).apply(
        lambda group: sum(
            np.power(group["true_return"], 2) 
            - np.power(group["true_return"] - group["pred_return"], 2)
        )
    )
    )
    return mean(c) / std(c)

In [40]:
import numpy as np
import datetime as dt

In [111]:
from evaluation_matrics import R_squared_OSXS


In [112]:
np.random.random((3,)) ** 2

array([0.46834916, 0.19331543, 0.01814334])

In [113]:
a = np.random.random((10,))

In [114]:
R_squared_OSXS(a, a*0.5 + 1)

0.75

In [119]:
CW_test(a, a*0.5 + 1)

/var/folders/nh/s3ft86ks12xd4qdptmv1pvtc0000gn/T/ipykernel_64490/1530711330.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  return np.mean(c) / np.std(c)


-inf

In [1]:
import pandas as pd 
import os


In [3]:
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit([[1, 2], [np.nan, 3], [7, 6]])

X = [[np.nan, 2], [6, np.nan], [7, 6]]
# imp.fit(X)
# print(imp.transform(X))
imp.transform(X)

array([[4.        , 2.        ],
       [6.        , 3.66666667],
       [7.        , 6.        ]])

## Import data

In [66]:
DATAROOT = "data/"

In [67]:
option_with_feature = pd.read_csv(os.path.join(DATAROOT, "option_with_nonsparse_features.csv"))

In [68]:
total_row_num = option_with_feature.shape[0]
option_with_feature = option_with_feature.dropna()
print(f"number of na rows: {total_row_num - option_with_feature.shape[0]}")

number of na rows: 1016067


In [70]:
total_row_num

3982036

In [69]:
option_with_feature["date_x"] = option_with_feature.date_x.apply(lambda x: dt.datetime.strptime(x.split()[0], "%Y-%m-%d")).copy()

In [71]:
option_with_feature.sample(3)

,Unnamed: 0,secid,date_x,exdate,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,...,MaxRet,PriceDelayRsq,PriceDelaySlope,PriceDelayTstat,ReturnSkew,ReturnSkew3F,VolMkt,zerotrade,zerotradeAlt1,date_y
302260,302260,106744,2000-08-31,2000-10-21 00:00:00,P,22.5,1.0,1.0625,3,2319,...,-0.064897,0.117214,1.001644,2.990750,-0.351963,-0.111258,-0.089172,2.444607e-08,3.998864e-08,2000-08-01
2519758,2519758,109942,2017-05-31,2017-07-21 00:00:00,C,70.0,0.7,0.8000,726,12,...,-0.019976,0.108427,-0.915030,-2.156536,-0.193380,-0.380172,-0.137936,1.499540e-08,1.462524e-08,2017-05-01
2953211,2953211,161291,2019-02-28,2020-01-17 00:00:00,C,105.0,3.7,4.0000,2,113,...,-0.056771,0.012893,0.459704,1.792312,-1.860529,-1.750176,-0.098604,2.206514e-08,1.862002e-08,2019-02-01


In [73]:
begin_date = dt.datetime.strptime("1996-01-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2000-12-31", "%Y-%m-%d")
training_data = option_with_feature[
    option_with_feature.date_x.apply(lambda x: x >= begin_date and x <= end_date )
]

In [74]:
begin_date = dt.datetime.strptime("2001-01-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2002-12-31", "%Y-%m-%d")
validation_data = option_with_feature[
    option_with_feature.date_x.apply(lambda x: x >= begin_date and x <= end_date )
]

In [75]:
begin_date = dt.datetime.strptime("2003-01-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2003-12-31", "%Y-%m-%d")
test_data = option_with_feature[
    option_with_feature.date_x.apply(lambda x: x >= begin_date and x <= end_date )
]

In [76]:
training_data.shape[0], validation_data.shape[0], test_data.shape[0]

(309930, 142846, 71364)

In [106]:
len(test_data.optionid), len(set(test_data.optionid))

(71364, 38039)

In [36]:
subsample = option_with_feature.sample(1000000)

In [27]:
subsample = subsample.dropna()

In [28]:
for i in (set(subsample.columns) - 
          {"Unnamed: 0", "secid", "date_x", "exdate", "date_y", "option_ret", "permno",
           "yyyymm", "date2", "sdate", "edate", 
           "cp_flag"}):
    print(f"'{i}'", end=", ")

'zerotrade', 'zerotradeAlt1', 'vega', 'optionid', 'IdioRisk', 'spotprice', 'Coskewness', 'High52', 'IdioVol3F', 'volume', 'PriceDelayRsq', 'theta', 'cfadj', 'MaxRet', 'adj_spot', 'Beta', 'best_bid', 'DolVol', 'days_no_trading', 'strike_price', 'VolMkt', 'forwardprice', 'ir_rate', 'impl_volatility', 'mid_price', 'BidAskSpread', 'best_offer', 'ReturnSkew', 'PriceDelayTstat', 'open_interest', 'delta', 'ReturnSkew3F', 'gamma', 'days_to_exp', 'IdioVolAHT', 'PriceDelaySlope', 

In [121]:
used_characteristics = ['volume', 'ReturnSkew', 'MaxRet', 'delta', 'PriceDelaySlope', 'strike_price', 'IdioVol3F', 'ReturnSkew3F', 'optionid', 'ir_rate', 'mid_price', 'forwardprice', 'zerotradeAlt1', 'theta', 'cfadj', 'zerotrade', 'best_bid', 'spotprice', 'VolMkt', 'IdioRisk', 'days_to_exp', 'PriceDelayTstat', 'High52', 'Coskewness', 'BidAskSpread', 'Beta', 'days_no_trading', 'open_interest', 'impl_volatility', 'PriceDelayRsq', 'IdioVolAHT', 'adj_spot', 'vega', 'gamma', 'best_offer', 'DolVol']

X_train = training_data[training_data.columns.difference(["option_ret"])]
y_train = training_data['option_ret']


X_test = test_data[test_data.columns.difference(["option_ret"])]
y_test = test_data['option_ret']

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y
# )

## Linear regression

In [78]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [79]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [80]:
from sklearn import linear_model

In [81]:
sum(y_train.isna())

0

In [122]:
regressor = linear_model.Lasso(alpha=0.1)
regressor.fit(X_train[used_characteristics], y_train)

Lasso(alpha=0.1)

In [123]:
y_pred = regressor.predict(X_test[used_characteristics])
mean_squared_error(y_test, y_pred)

178.6995144070895

In [124]:
r2_score(y_test, y_pred)

-0.0011067996607445618

In [125]:
R_squared_OSXS(y_test, y_pred)

-0.0003612560350199079

In [ ]:
# CW test

In [129]:
true_pred_return = pd.DataFrame(
    {
        "optionid": X_test.optionid,
        "time": X_test.date_x,
        "true_return": y_test,
        "pred_return": y_pred
    }
)

In [ ]:
# DW test

In [158]:
true_pred_1vs2_return = pd.DataFrame(
    {
        "optionid": X_test.optionid,
        "time": X_test.date_x,
        "true_return": y_test,
        "pred_return1": regressor.predict(X_test[used_characteristics]),
        "pred_return2": est.predict(X_test[used_characteristics])
    }
)

In [159]:
d = 1 / true_pred_1vs2_return.shape[0] * true_pred_1vs2_return.groupby(["optionid", "time"]).apply(
    lambda group:
    sum(
        np.power(group["true_return"] - group["pred_return1"], 2)
    - np.power(group["true_return"] - group["pred_return2"], 2)
    )
)

In [160]:
np.mean(d) / np.std(d)

0.037391263300535625

In [136]:
np.mean(c)

-2.781837086300935e-06

## Emsemble models

In [86]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor

In [87]:
import sklearn
sklearn.__version__

'1.0.2'

In [104]:
est = GradientBoostingRegressor(
    n_estimators=100, random_state=0,
    loss='huber',
    verbose=1
).fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           0.3613            3.46m
         2           0.3600            3.41m
         3           0.3591            3.33m
         4           0.3579            3.29m
         5           0.3562            3.26m
         6           0.3549            3.21m
         7           0.3541            3.17m
         8           0.3532            3.13m
         9           0.3525            3.10m
        10           0.3513            3.07m
        20           0.3466            2.75m
        30           0.3441            2.39m
        40           0.3421            2.03m
        50           0.3403            1.69m
        60           0.3391            1.35m
        70           0.3381            1.01m
        80           0.3370           40.36s
        90           0.3357           20.20s
       100           0.3346            0.00s


In [ ]:
y_pred = est.predict(X_test)

In [101]:
mean_squared_error(y_test, y_pred)

178.50648699283414

In [102]:
r2_score(y_test, y_pred)

-2.5425390796751657e-05

In [103]:
R_squared_OSXS(y_test, est.predict(X_test))

-2.4875359702081212e-05

In [120]:
X_test.sample()

,volume,ReturnSkew,MaxRet,delta,PriceDelaySlope,strike_price,IdioVol3F,ReturnSkew3F,optionid,ir_rate,...,days_no_trading,open_interest,impl_volatility,PriceDelayRsq,IdioVolAHT,adj_spot,vega,gamma,best_offer,DolVol
515971,24,0.8303,-0.057629,0.212497,0.038668,22.5,-0.021264,0.614638,20960240,0.012316,...,0,181,0.465872,0.010661,-0.037061,15.07,3.69867,0.048886,0.7,-7.570561
